In [4]:
# La neta que weba estar instalando a cada rato estos chingados paquetes
# pero al menos solo los descarga una vez y a la siguiente solo los copia 
# a la carpeta de trabajo del virtualenv
%pip install numpy
%pip install pandas
%pip install tensorflow
%pip install tensorflow_decision_forests
%pip install cuda-python # Esto es para que tensorflow pueda usar la GPU de nvidia serie RTX2000 o superior (si no lo tienes, no lo instales)

import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf

print( f"Encontrado TF-DF {tfdf.__version__}" )

Encontrado TF-DF 1.5.0


In [5]:
train_df = pd.read_csv( "train.csv" )
serving_df = pd.read_csv( "test.csv" )

train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [6]:
def preprocess( df ):
  df = df.copy()

  def normalize_name( x ):
    return " ".join( [ v.strip( ",()[].\"'") for v in x.split( " " )])

  def ticket_number( x ):
    return x.split( " " )[-1]

  def ticket_item( x ):
    items = x.split( " " )
    if len( items ) == 1:
      return "NONE"
    return "_".join( items[ 0:-1 ] )

  df["Name"] = df[ "Name" ].apply( normalize_name )
  df[ "Ticket_number" ] = df[ "Ticket" ].apply( ticket_number )
  df[ "Ticket_item" ] = df[ "Ticket" ].apply( ticket_item )
  return df

preprocessed_train_df = preprocess( train_df )
preprocessed_serving_df = preprocess( serving_df )

preprocessed_train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE


In [7]:
input_features = list( preprocessed_train_df.columns )
input_features.remove( "Ticket" )
input_features.remove( "PassengerId" )
input_features.remove( "Survived" )

print( f"Input features: {input_features}" )

Input features: ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


# Se convierte el Dataset de Pandas a Tensorflow

In [10]:
def tokenize_names( features, labels = None ):
  """Divide the names into tokens. TF-DF can consume text tokens natively."""
  features[ "Name" ] = tf.strings.split( features[ "Name" ] )
  return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset( preprocessed_train_df, label = "Survived" ).map( tokenize_names )
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset( preprocessed_serving_df).map( tokenize_names )

# Se entrena el modelo con los parametros por default

In [11]:
model = tfdf.keras.GradientBoostedTreesModel(
  verbose = 1, # Use verbose=0 for silent training.
  features = [tfdf.keras.FeatureUsage( name = n ) for n in input_features ],
  exclude_non_specified_features = True, # Only use the features in "features".
  random_seed = 1234, # For determinism.
)
model.fit( train_ds )

self_evaluation = model.make_inspector().evaluation()
print( f"Accuracy: {self_evaluation.accuracy} Loss: {self_evaluation.loss}" )

Use /tmp/tmp_804nkvc as temporary training directory
Reading training dataset...


[WARNING 23-08-27 11:06:48.7624 CST gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-27 11:06:48.7624 CST gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-27 11:06:48.7624 CST gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


Training dataset read in 0:00:02.950081. Found 891 examples.
Training model...
Model trained in 0:00:00.170875
Compiling model...


[INFO 23-08-27 11:06:51.8937 CST kernel.cc:1243] Loading model from path /tmp/tmp_804nkvc/model/ with prefix 2426ead2bbb04e90
[INFO 23-08-27 11:06:51.8968 CST abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-08-27 11:06:51.8968 CST kernel.cc:1075] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.
Accuracy: 0.8260869383811951 Loss: 0.8608942627906799


# Se entrena el modelo con los parametros por default mejorados

In [13]:
model = tfdf.keras.GradientBoostedTreesModel(
  verbose = 1, # Use verbose=0 for silent training.
  features = [ tfdf.keras.FeatureUsage( name = n ) for n in input_features ], 
  exclude_non_specified_features = True, # Only use the features in "features".

  # num_trees = 2000, # uncomment to train 2000 trees

  # Only for GBT (Gradient Boosted Trees) models:
  # A bit slower, but great to understand the model.
  # compute_permutation_variable_importance = True,

  # Change the default hyper-parameters
  # hyperparameter_template = "benchmark_rank1@v1",

  # num_trees = 1000, 
  # tuner = tuner, 

  min_examples = 1, 
  categorical_algorithm = "RANDOM", 
  # max_depth = 4, 
  shrinkage = 0.05, 
  # num_candidate_attributes_ratio = 0.2, 
  split_axis = "SPARSE_OBLIQUE", 
  sparse_oblique_normalization = "MIN_MAX", 
  sparse_oblique_num_projections_exponent = 2.0, 
  num_trees = 2000, 
  # validation_ratio = 0.0, 
  random_seed = 1234, # For determinism.
)

model.fit( train_ds )

self_evaluation = model.make_inspector().evaluation()
print( f"Accuracy: {self_evaluation.accuracy} Loss: {self_evaluation.loss}" )

Use /tmp/tmpec49sd40 as temporary training directory
Reading training dataset...


[WARNING 23-08-27 11:17:34.6371 CST gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-27 11:17:34.6371 CST gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-27 11:17:34.6372 CST gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


Training dataset read in 0:00:00.219294. Found 891 examples.
Training model...
Model trained in 0:00:00.266357
Compiling model...
Model compiled.
Accuracy: 0.782608687877655 Loss: 1.060815453529358


[INFO 23-08-27 11:17:35.1240 CST kernel.cc:1243] Loading model from path /tmp/tmpec49sd40/model/ with prefix 2a42aa66f1524c48
[INFO 23-08-27 11:17:35.1287 CST decision_forest.cc:660] Model loaded with 42 root(s), 2212 node(s), and 10 input feature(s).
[INFO 23-08-27 11:17:35.1287 CST abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-08-27 11:17:35.1287 CST kernel.cc:1075] Use fast generic engine
